In [1]:
from ipywidgets import interact
import time

# for building application
from sequence.app.request_app import RequestApp
from sequence.message import Message

# for building network
from sequence.topology.router_net_topo import RouterNetTopo

In [2]:
START_TIME = 1e12

class TeleportMessage(Message):
    def __init__(self, msg_type, receiver, memo_name: str):
        super().__init__(msg_type, receiver)
        self.memo_name = memo_name

class TeleportApp(RequestApp):
    def __init__(self, node, name, other_name):
        super().__init__(node)
        self.name = name
        self.other_name = other_name
        node.protocols.append(self)
        self.memos_to_measure = {}
        
        # collect metrics:
        self.latency = 0
        self.count = 0
        
    def get_memory(self, info: "MemoryInfo") -> None:
        """Method to receive entangled memories.

        Will check if the received memory is qualified.
        If it's a qualified memory, the application sets memory to RAW state
        and release back to resource manager.
        The counter of entanglement memories, 'memory_counter', is added.
        Otherwise, the application does not modify the state of memory and
        release back to the resource manager.

        Args:
            info (MemoryInfo): info on the qualified entangled memory.
        """

        if info.state != "ENTANGLED":
            return

        if info.index in self.memo_to_reserve:
            reservation = self.memo_to_reserve[info.index]
            
            if info.remote_node == reservation.responder and info.fidelity >= reservation.fidelity:
                # we are initiator, and want to teleport qubit
                print("node {} memory {} entangled with other memory {}".format(
                    self.node.name, info.index, info.remote_memo))
                
                # record metrics
                if self.count == 0:
                    self.latency = (self.node.timeline.now() - START_TIME) * 1e-12
                self.count += 1
                
                # send message to other node
                message = TeleportMessage(None, self.other_name, info.remote_memo)
                self.node.send_message(self.responder, message)
                
                # reset local memory
                self.node.resource_manager.update(None, info.memory, "RAW")
                
            elif info.remote_node == reservation.initiator and info.fidelity >= reservation.fidelity:
                # we are responder, and want to receive qubit
                # need to wait on message from sender to correct entanled memory
                self.memos_to_measure[info.memory.name] = info.memory
                
    def received_message(self, src, message):
        memo_name = message.memo_name
        
        print("node {} received teleportation message for memory {}".format(
            self.node.name, memo_name))
        
        # reset local memory
        memory = self.memos_to_measure.pop(memo_name)
        self.node.resource_manager.update(None, memory, "RAW")

In [3]:
def set_parameters(topology, simulation_time, attenuation):
    """
    simulation_time: duration of simulation time (s)
    attenuation: attenuation on quantum channels (db/m)
    """
    
    PS_PER_S = 1e12
    
    # set timeline stop time
    topology.get_timeline().stop_time = (simulation_time * PS_PER_S)
    
    # set memory parameters
    MEMO_FREQ = 2e3
    MEMO_EXPIRE = 0
    MEMO_EFFICIENCY = 1
    MEMO_FIDELITY = 0.9349367588934053
    for node in topology.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        memory_array = node.get_components_by_type("MemoryArray")[0]
        memory_array.update_memory_params("frequency", MEMO_FREQ)
        memory_array.update_memory_params("coherence_time", MEMO_EXPIRE)
        memory_array.update_memory_params("efficiency", MEMO_EFFICIENCY)
        memory_array.update_memory_params("raw_fidelity", MEMO_FIDELITY)

    # set detector parameters
    DETECTOR_EFFICIENCY = 0.9
    DETECTOR_COUNT_RATE = 5e7
    DETECTOR_RESOLUTION = 100
    for node in topology.get_nodes_by_type(RouterNetTopo.BSM_NODE):
        bsm = node.get_components_by_type("SingleAtomBSM")[0]
        bsm.update_detectors_params("efficiency", DETECTOR_EFFICIENCY)
        bsm.update_detectors_params("count_rate", DETECTOR_COUNT_RATE)
        bsm.update_detectors_params("time_resolution", DETECTOR_RESOLUTION)
        
    # set entanglement swapping parameters
    SWAP_SUCC_PROB = 0.90
    SWAP_DEGRADATION = 0.99
    for node in topology.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        node.network_manager.protocol_stack[1].set_swapping_success_rate(SWAP_SUCC_PROB)
        node.network_manager.protocol_stack[1].set_swapping_degradation(SWAP_DEGRADATION)
        
    # set quantum channel parameters
    ATTENUATION = attenuation
    QC_FREQ = 1e11
    for qc in topology.qchannels:
        qc.attenuation = ATTENUATION
        qc.frequency = QC_FREQ

In [4]:
def test(sim_time=1.5, qc_atten=1e-5):
    """
    sim_time: duration of simulation time (s)
    qc_atten: quantum channel attenuation (dB/km)
    """
    
    network_config = "star_network.json"
    
    # here, we make a new topology using the configuration JSON file.
    # we then modify some of the simulation parameters of the network.
    network_topo = RouterNetTopo(network_config)
    set_parameters(network_topo, sim_time, qc_atten)
    
    # get two end nodes and create application
    start_node_name = "router1"
    end_node_name = "router2"
    node1 = node2 = None

    for router in network_topo.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        if router.name == start_node_name:
            node1 = router
        elif router.name == end_node_name:
            node2 = router
            
    start_app_name = "start_app"
    end_app_name = "end_app"
    
    start_app = TeleportApp(node1, start_app_name, end_app_name)
    end_app = TeleportApp(node2, end_app_name, start_app_name)
        
    # run the simulation
    tl = network_topo.get_timeline()
    tl.show_progress = False
    tl.init()
    
    start_app.start(end_node_name, START_TIME, 2e12, 10, 0.9)
    tick = time.time()
    tl.run()
    
    print("\n")
    print("Execution time: {:.3f} seconds".format(time.time() - tick))
    
    print("\n")
    print("Latency: {:.3f} s".format(start_app.latency))
    print("Number of entangled memories:", start_app.count)
    print("Average throughput: {:.3f} pairs/s".format(
        start_app.count / (sim_time - (START_TIME * 1e-12))))

In [11]:
interact(test, sim_time=(1, 3, 0.1), qc_atten=[0, 1e-5, 2e-5])

interactive(children=(FloatSlider(value=1.5, description='sim_time', max=3.0, min=1.0), Dropdown(description='…

<function __main__.test(sim_time=1.5, qc_atten=1e-05)>